# Calculating ion solutions

## Configuration

Hydrogen only. Requires `dilute-lte` excitation and `nebular` ionization to provide the first approximation for the solver

In [1]:
from tardis.io.configuration.config_reader import Configuration

config = Configuration.from_yaml("../../../../../tardis/tardis/plasma/tests/data/plasma_base_test_config.yml")

config.model.abundances.He = 0
config.model.abundances.H = 1

config.plasma.excitation = "dilute-lte"
config.plasma.ionization = "nebular"

Iterations:          0/? [00:00<?, ?it/s]

Packets:             0/? [00:00<?, ?it/s]

## Atomic data

Requires photoionization cross section data

In [2]:
from copy import deepcopy

from tardis.io.atom_data import AtomData
from tardis.model.base import SimulationState

atom_data = AtomData.from_hdf('/tardis-regression-data/atom_data/nlte_atom_data/TestNLTE_He_Ti.h5')

atom_data2 = deepcopy(atom_data)

atom_data2.prepare_atom_data([1], "macroatom", [], [(1, 0)])

sim_state = SimulationState.from_config(config, atom_data=atom_data)

Number of density points larger than number of shells. Assuming inner point irrelevant


In [3]:
import astropy.units as u
import numpy as np

from tardis.plasma.electron_energy_distribution import (
    ThermalElectronEnergyDistribution,
)
from tardis.plasma.radiation_field import (
    DilutePlanckianRadiationField,
)

rad_field = DilutePlanckianRadiationField(np.ones(20) * 10000 * u.K, dilution_factor=np.ones(20) * 0.5)

electron_dist = ThermalElectronEnergyDistribution(0, np.ones(20) * 10000 * u.K, np.ones(20) * 2e9 * u.cm**-3)

# Create a plasma

In [4]:
from tardis.plasma.assembly.base import PlasmaSolverFactory

plasma_solver_factory = PlasmaSolverFactory(atom_data, config)
plasma_solver_factory.prepare_factory([1],"tardis.plasma.properties.property_collections" )
plasma = plasma_solver_factory.assemble(sim_state.elemental_number_density, rad_field, sim_state.time_explosion, electron_dist.number_density, link_t_rad_t_electron=1)

Zeta_data missing - replaced with 1s. Missing ions: []
Zeta_data missing - replaced with 1s. Missing ions: []


# Create LTE properties

In [5]:
from tardis.plasma.properties.ion_population import IonNumberDensity


class LTEIonNumberDensity(IonNumberDensity):
    outputs = ('lte_ion_number_density', )
    latex_name = ('N_{i,j}^*',)

    def calculate(self, phi_Te, lte_partition_function_Te, number_density, electron_densities, block_ids, ion_threshold):
        return self.calculate_with_n_electron(
            phi_Te, lte_partition_function_Te, number_density, electron_densities, block_ids, ion_threshold)

In [6]:
from tardis.plasma.properties.level_population import (
    LevelNumberDensity,
)


class LTELevelNumberDensity(LevelNumberDensity):
    outputs = ('lte_level_number_density',)
    latex_name = ('N_{i,j,k}^*',)

    def _calculate_dilute_lte(self, lte_level_boltzmann_factor_Te, lte_ion_number_density,
                              levels, lte_partition_function_Te):
        return super()._calculate_dilute_lte(
            lte_level_boltzmann_factor_Te, lte_ion_number_density, levels, lte_partition_function_Te)

In [7]:
lte_ion_number_density = LTEIonNumberDensity(plasma,
                                             electron_densities=plasma.electron_densities
                                             ).calculate(
                                                 plasma.thermal_phi_lte,
                                                 plasma.thermal_lte_partition_function,
                                                 sim_state.elemental_number_density,
                                                 plasma.electron_densities,
                                                 None, 1e-20)[0]

In [8]:
lte_level_number_density = LTELevelNumberDensity(plasma).calculate(
                                                     plasma.thermal_lte_level_boltzmann_factor,
                                                     lte_ion_number_density,
                                                     plasma.levels,
                                                     plasma.thermal_lte_partition_function)

# Initialize ionization solvers

In [9]:
from tardis.plasma.equilibrium.rate_matrix import IonRateMatrix
from tardis.plasma.equilibrium.rates import (
    AnalyticPhotoionizationRateSolver,
    CollisionalIonizationRateSolver,
)

photoionization_rate_solver = AnalyticPhotoionizationRateSolver(atom_data2.photoionization_data)

collisional_rate_solver = CollisionalIonizationRateSolver(atom_data2.photoionization_data)

ion_rate_matrix_solver = IonRateMatrix(photoionization_rate_solver, collisional_rate_solver)

## Optional: solve for radiative ionization rates

In [10]:
photoion_rates, recomb_rates = photoionization_rate_solver.solve(
    rad_field,
    electron_dist,
    lte_level_number_density.loc[lte_level_number_density.index.get_level_values('ion_number') < 1],
    plasma.level_number_density.loc[plasma.level_number_density.index.get_level_values('ion_number') < 1],
    lte_ion_number_density.loc[lte_ion_number_density.index.get_level_values('ion_number') >= 1],
    plasma.ion_number_density.loc[plasma.ion_number_density.index.get_level_values('ion_number') >= 1],)

## Optional: solve for collisional ionization rates

In [12]:
saha_factor = lte_level_number_density.loc[lte_level_number_density.index.get_level_values('ion_number') < 1] / (
            lte_ion_number_density.loc[lte_ion_number_density.index.get_level_values('ion_number') >= 1].values
            * electron_dist.number_density.value
        )

coll_ionization_rate, coll_recomb_rate = collisional_rate_solver.solve(
    electron_dist,
    saha_factor,
)

## Optional: solve for the ionization rate matrix

In [14]:
ion_rate_matrix = ion_rate_matrix_solver.solve(
    rad_field,
    electron_dist,
    lte_level_number_density.loc[lte_level_number_density.index.get_level_values('ion_number') < 1],
    plasma.level_number_density.loc[plasma.level_number_density.index.get_level_values('ion_number') < 1],
    lte_ion_number_density.loc[lte_ion_number_density.index.get_level_values('ion_number') >= 1],
    plasma.ion_number_density.loc[plasma.ion_number_density.index.get_level_values('ion_number') >= 1],
    charge_conservation=True)

Ionization rate matrix for cell 0. First row is charge conservation. Second row is number conservation.

In [15]:
ion_rate_matrix[0].values

array([array([[ 0.00000000e+00,  1.00000000e+00, -1.00000000e+00],
              [ 1.00000000e+00,  1.00000000e+00,  0.00000000e+00],
              [ 1.59345585e+17, -4.76114211e+07,  0.00000000e+00]])],
      dtype=object)

# Solve ion population

In [16]:
from tardis.plasma.equilibrium.ion_populations import IonPopulationSolver

solver = IonPopulationSolver(ion_rate_matrix_solver, [(1, 0), (1,1)])

ion_pops = solver.solve(
    rad_field,
    electron_dist,
    lte_level_number_density,
    plasma.level_number_density,
    lte_ion_number_density,
    plasma.ion_number_density,
    charge_conservation=True)

In [17]:
ion_pops

0             1             2   \
atomic_number ion_number                                             
1             0           3.747555e-10  3.810791e-10  3.890471e-10   
              1           1.000000e+00  1.000000e+00  1.000000e+00   

                                    3             4             5   \
atomic_number ion_number                                             
1             0           3.990015e-10  4.113389e-10  4.265170e-10   
              1           1.000000e+00  1.000000e+00  1.000000e+00   

                                    6             7             8   \
atomic_number ion_number                                             
1             0           4.450609e-10  4.675712e-10  4.947310e-10   
              1           1.000000e+00  1.000000e+00  1.000000e+00   

                                    9             10            11  \
atomic_number ion_number                                             
1             0           5.273145e-10  5.661958e-10  6.123583e-10   
              1           1.000000e+00  1.000000e+00  1.000000e+00   

                                    12            13            14  \
atomic_number ion_number                                             
1             0           6.669046e-10  7.310669e-10  8.062184e-10   
              1           1.000000e+00  1.000000e+00  1.000000e+00   

                                    15            16            17  \
atomic_number ion_number                                             
1             0           8.938847e-10  9.957564e-10  1.113702e-09   
              1           1.000000e+00  1.000000e+00  1.000000e+00   

                                    18            19  
atomic_number ion_number                              
1             0           1.249782e-09  1.406263e-09  
              1           1.000000e+00  1.000000e+00